# Scrapping twitter

Fem servir la API no oficial [twscrape](https://github.com/vladkens/twscrape)

Aquest notebook és només un esquema general, **vegeu els 3 projectes d'exemple** per veure algunes de les possibilitats reals:
- RAE:
    - la query: `"#dudaRAE"` per un seguit de franges temporals, fent servir `since:` i `until:`
    - Es crea un diccionari de respostes de l'API segons aquestes dates
    - Processat de dades i obtenció de respostes als tweets obtinguts
- la_revuelta: similar a l'anterior, dues queries: `"#LaRevuelta"` i `"@LaRevueltaTVE"` per un seguit de franges temporals, fent servir `since:` i `until:`
- far_right_politics: gran volum de tweets segons una llista d'usuaris (no pública)

## Imports

In [ ]:
import asyncio
import pickle
import sys
from pathlib import Path

import pandas as pd
from twscrape import API, gather
from twscrape.logger import set_log_level

set_log_level("DEBUG")
api = API()  # inicialitza l'API amb la base de dades de comptes predeterminada `accounts.db`

pd.set_option("display.max_colwidth", None)

## Comptes

El gran benefici d'aquesta API és que permet afegir diversos comptes i els va rotant quan arriben al límit cada 15 minuts (pot anar canviant), a més un cop ha passat el temps d'espera automàticament continua. Per tant, si volem moltes piulades senzillament serà qüestió de deixar el programa executant durant hores.

Si feu un ús intensiu de l'eina, recomano no utilitzar comptes personals.

In [ ]:
async def setup_accounts():
    cookies = "abc=12; ct0=xyz"
    # Exemple de la documentació. Jo sempre he fet sevir pass3 == mail_pass3 i sembla funcionar
    await api.pool.add_account("user3", "pass3", "u3@mail.com", "mail_pass3", cookies=cookies)
    await api.pool.login_all()


await setup_accounts()

Podem veure els comptes guardats a la base de dades amb sqlite

```python
import sqlite3

cnx = sqlite3.connect("accounts.db")
pd.read_sql_query("SELECT * FROM accounts", cnx)
```

Recomano afegir els comptes d'un en un, ja que és possible que donin problemes.
Pot ser que aquest procediment ajudi a solucionar-ho (a vegades cal executar-ho diverses vegades):
Font: https://github.com/vladkens/twscrape/issues/214

```python
# 1)
! twscrape del_accounts user3

# 2) tornar a executar setup_accounts()
await setup_accounts()

# 3)
! twscrape relogin user3
```

# Cerca

Podem buscar de diverses maneres, la que ens dona més opcions és `api.search`, ja que permet buscar amb la mateixa sintaxi que ho faríem a twitter (vegeu https://github.com/igorbrigadir/twitter-advanced-search, però també pot haver canviat).

Així doncs, podem buscar per _hashtag_, per compte d'usuari, per paraula clau... A més podem limitar la cerca a unes dates determinades o fer servir certs filtres.

La sintaxi bàscia és:
```python
tweets = await gather(api.search(query, limit=10))
```

El límit és el nombre de tweets que ens agradaria obtenir, si no n'hi ha tants serà menor, però també pot ser major per com funciona l'API de twitter.

La query pot ser:
```python
query = "#un_hashtag"
query = "@mencio_usuari"
query = "from:usuari"
query = "paraules concretes"
```

`tweets` és un objecte amb molta informació. És recomanable guardar-lo per si la necessitem en un futur. Un cop guardat podem transformar l'objecte a un `DataFrame` de `pandas` per treballar-hi millor.

In [ ]:
# guardar tweets
with open("tweets.pkl", "wb") as f:
    pickle.dump(tweets, f)

# Carregar tweets guardats
# with open("tweets.pkl", "rb") as f:
#     tweets = pickle.load(f)

## Processat de dades

Seleccionem la informació que ens interessa. Per triar què volem és interessant observar almenys unes quants tweets en el seu format original. Per exemple fent:
```python
tweets[0].dict()
```

In [ ]:
# 1) Seleccionem la informació que volem
def parse_tweet(tweet):
    return {
        "id": tweet.id,
        "username": tweet.user.username,
        "content": tweet.rawContent,
    }


# 2) Transformem el diccionari a un DataFrame
def tweets2df(tweets):
    parsed_data = []
    for tweet in tweets:
        tweet_data = parse_tweet(tweet)
        parsed_data.append(tweet_data)
    df = pd.DataFrame(parsed_data)
    return df

In [ ]:
df = get_df(tweets)
df.to_csv("tweets.csv", index=False)